In [37]:
import requests
import pandas as pd
from funSentiments import *
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')
sia = SentimentIntensityAnalyzer()

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /home/jorge/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [2]:
df = pd.read_csv('input/All-seasons.csv')

In [3]:
df.head()

,Season,Episode,Character,Line
0,10,1,Stan,"You guys, you guys! Chef is going away. \n"
1,10,1,Kyle,Going away? For how long?\n
2,10,1,Stan,Forever.\n
3,10,1,Chef,I'm sorry boys.\n
4,10,1,Stan,"Chef said he's been bored, so he joining a gro..."


## Get episode of a season you want

In [18]:
df_epi2= episode(3,2)

In [25]:
df_epi14= episode(7,14)

In [26]:
df_epi14.head()

,Character,Line
63179,Stan,"All right, you guys, it's first down. I'll hik..."
63180,Cartman,Right. What are we playing again?\n
63181,Stan,Football.\n
63182,Cartman,Got it. \n
63183,Stan,"Okay. 23! Blue, 23! Set hut! Set hut! \n"


## Add all Characters in Database

It will add just new characters

In [27]:
cha_epi2 = get_character(df_epi14)

In [28]:
cha_epi14= get_character(df_epi14)

In [23]:
for x in cha_epi2:
    res = requests.get(f'http://localhost:3000/chat/create/{x}')
print(res)
res.json()

<Response [200]>


'New user create! user_name:The Three Spirits, user_id: 5f0ca7984c36241d385b8990 '

In [29]:
for x in cha_epi14:
    res = requests.get(f'http://localhost:3000/chat/create/{x}')
print(res)
res.json()

<Response [200]>


'New user create! user_name:Goth Voice, user_id: 5f0ca7b84c36241d385b89a9 '

## Add episode to database

In [30]:
participants= cha_epi2
dicti={"participants":participants, "chat_name": "Season 3 Episode 2" }

In [31]:
#http://localhost:3000/chat/create

url = "http://localhost:3000/chat/create"
res = requests.get(url, params= dicti)
print(res)
res.json()

<Response [200]>


'New chat create! Conversation id: 5f0ca8cd4c36241d385b89aa'

In [32]:
participants= cha_epi14
dicti={"participants":participants, "chat_name": "Season 7 Episode 14" }

In [33]:
#http://localhost:3000/chat/create

url = "http://localhost:3000/chat/create"
res = requests.get(url, params= dicti)
print(res)
res.json()

<Response [200]>


'New chat create! Conversation id: 5f0ca8de4c36241d385b89ab'

## Add all dialogues of the episode

In [35]:
conversation_id= "5f0ca8cd4c36241d385b89aa"
url = f"http://localhost:3000/chat/{conversation_id}/addmessage"

for index, row in df_epi2.iterrows():
    message={"conversation_id":f'{conversation_id}', "user_name":f'{row["Character"]}' , "message":f'{row["Line"]}'}
    res = requests.post(url, data=message)
    
print(res)
res.json()

<Response [200]>


'New Message added. Id message: 5f0ca98a4c36241d385b8a58'

In [36]:
conversation_id= "5f0ca8de4c36241d385b89ab"
url = f"http://localhost:3000/chat/{conversation_id}/addmessage"

for index, row in df_epi14.iterrows():
    message={"conversation_id":f'{conversation_id}', "user_name":f'{row["Character"]}' , "message":f'{row["Line"]}'}
    res = requests.post(url, data=message)
    
print(res)
res.json()

<Response [200]>


'New Message added. Id message: 5f0ca9b34c36241d385b8b78'

# Sentiments

## Analize sentiments by all character and message in an episode

In [39]:
#http://localhost:3000/chat/<conversation_id>/sentiment

conversation_id= "5f0ca8de4c36241d385b89ab"

url = f'http://localhost:3000/chat/{conversation_id}/sentiment'
res = requests.get(url)
print(res)
data=res.json()
data[:4]

<Response [200]>


[{'name': 'Stan',
  'message': 'All right, you guys, it\'s first down. I\'ll hike the ball on the third "set hut!" Kyle and Cartman, go deep post out. Kenny, run a slant down the middle. Butters, be ready for the screen.\n',
  'sentiments': {'neg': 0.0, 'neu': 0.924, 'pos': 0.076, 'compound': 0.4199}},
 {'name': 'Cartman',
  'message': 'Right. What are we playing again?\n',
  'sentiments': {'neg': 0.0, 'neu': 0.735, 'pos': 0.265, 'compound': 0.2023}},
 {'name': 'Stan',
  'message': 'Football.\n',
  'sentiments': {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}},
 {'name': 'Cartman',
  'message': 'Got it. \n',
  'sentiments': {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}}]

## Analyze sentiments of specific character by messages in an episode

In [40]:
#http://localhost:3000/chat/<conversation_id>/sentiment/<name>

conversation_id= "5f0ca8de4c36241d385b89ab"
character= "Cartman"

url = f'http://localhost:3000/chat/{conversation_id}/sentiment/{character}'
res = requests.get(url)
print(res)
data= res.json()
data[:4]

<Response [200]>


[{'name': 'Cartman',
  'message': 'Right. What are we playing again?\n',
  'sentiments': {'neg': 0.0, 'neu': 0.735, 'pos': 0.265, 'compound': 0.2023}},
 {'name': 'Cartman',
  'message': 'Got it. \n',
  'sentiments': {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}},
 {'name': 'Cartman',
  'message': 'What-ever, Bebe! Like Stan really cares! Just get out of our football game, you stupid skank!\n',
  'sentiments': {'neg': 0.147,
   'neu': 0.564,
   'pos': 0.289,
   'compound': 0.5055}},
 {'name': 'Cartman',
  'message': 'Yeh-heah, right!\n',
  'sentiments': {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}}]

## Analyze total sentiments of a character in an episode

In [42]:
#http://localhost:3000/chat/<conversation_id>/sentiment/<name>/total

conversation_id= "5f0ca8de4c36241d385b89ab"
character= "Cartman"

url = f'http://localhost:3000/chat/{conversation_id}/sentiment/{character}/total'
res = requests.get(url)
print(res)
res.json()

<Response [200]>


'Cartman has 0.149 of negativity, 0.7246 of neutral and 0.1263 of positivity in this episode'

## Analyze total sentiments of a character in all seasons

In [49]:
#http://localhost:3000/chat/sentiment/<name>

character= "Cartman"

url = f'http://localhost:3000/chat/sentiment/{character}'
res = requests.get(url)
print(res)
res.json()

<Response [200]>


'Cartman has 0.1519 of negativity, 0.7641 of neutral  in all the episodes'

## Compare 2 character sentiments

In [50]:
#http://localhost:3000/chat/sentiment/<name1>/<name2>

character1= "Cartman"
character2= "Stan"

url = f'http://localhost:3000/chat/sentiment/{character1}/{character2}'
res = requests.get(url)
print(res)
res.json()

<Response [200]>


'Cartman has 0.1519 of negativity, 0.7641 of neutral and 0.1375 and Stan has 0.0934 of negativity, 0.769 of neutral and 0.1375 in all the episodes'

# Recomendations

Find similar characters to the one you choose

In [53]:
#http://localhost:3000/recomendation/<name>

character="Cartman"

url = f"http://localhost:3000/recomendation/{character}"
res = requests.get(url)
print(res)
res.json()

<Response [200]>


'Cartman is similar to Kyle, Stan and Butters '